In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random

2023-03-04 00:00:16.805427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
meta_df = pd.read_csv('../train_val_test_split/train_val_test_GoogleAudioSet.csv', index_col=0)
meta_df

,file,source,category,weight,fold
0,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,0
1,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8
2,../data/interim/GoogleAudioSet_unbalanced_list...,Google_nature,0,1,5
3,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,1
4,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,1
...,...,...,...,...,...
13662,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,5
13663,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,3
13664,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8
13665,../data/interim/GoogleAudioSet_unbalanced_list...,Google_city,1,1,8


In [3]:
def load_mps_pkl(filename, mps_ind):
    import pickle
    import pandas as pd
    # open a file, where you stored the pickled data
    file = open(filename, 'rb')

    # dump information to that file
    output = pickle.load(file)
    
    mps = output[mps_ind]
    wt = output['wt']
    mps = mps[:,wt<100].reshape(-1)

    # close the file
    file.close()
    return mps

In [4]:
mps_raw_list = []
mps_bg_list = []
mps_fg_list = []
for index, row in meta_df.iterrows():
    mps_raw_list.append(load_mps_pkl(row['file'], 'mps'))
    mps_bg_list.append(load_mps_pkl(row['file'], 'mps_bg'))
    mps_fg_list.append(load_mps_pkl(row['file'], 'mps_fg'))

In [5]:
from itertools import compress
train_raw = np.stack(compress(mps_raw_list, meta_df['fold']<8))
valid_raw = np.stack(compress(mps_raw_list, meta_df['fold']==8))

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if await self.run_code(code, result, async_=asy):


In [6]:
from lazypredict.Supervised import LazyClassifier
# from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

transformer = Pipeline(steps=[
    ('PCA', PCA(n_components = 10, random_state=23)),
    ('scaler', StandardScaler())
])
transformer.fit(train_raw)
train_raw_pca = transformer.transform(train_raw)
valid_raw_pca = transformer.transform(valid_raw)

y_train = meta_df.loc[meta_df['fold']<8, 'category']
y_valid = meta_df.loc[meta_df['fold']==8, 'category']

In [7]:
train_raw_pca.shape

(11933, 10)

In [8]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(train_raw_pca, valid_raw_pca, y_train, y_valid)
models

100%|███████████████████████████████████████████| 29/29 [01:03<00:00,  2.20s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.63,0.63,0.63,0.63,7.35
NearestCentroid,0.63,0.63,0.63,0.63,0.09
AdaBoostClassifier,0.63,0.63,0.63,0.63,1.50
LinearSVC,0.63,0.63,0.63,0.63,0.76
LogisticRegression,0.62,0.62,0.62,0.62,0.02
CalibratedClassifierCV,0.62,0.62,0.62,0.62,3.16
LinearDiscriminantAnalysis,0.62,0.62,0.62,0.62,0.04
RidgeClassifierCV,0.62,0.62,0.62,0.62,0.03
RidgeClassifier,0.62,0.62,0.62,0.62,0.02
